In [ ]:
%matplotlib inline

import numpy as np
import pylab as plt
import seaborn as sns
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from sklearn.metrics import confusion_matrix

# No longer needed
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize,
                          cmap=plt.cm.Blues):

    cm = confusion_matrix(y_true, y_pred)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
nG, nS, nQ= 30000, 15000, 15000
x_tr1= pd.concat([pd.read_csv('{}[0-15].csv'.format(i))[:j] for i,j in {'GALAXY':nG,'STAR':nS,'QSO':nQ}.items()])
x_tr= x_tr1.sample(frac=1).reset_index(drop=True)

frames1= [pd.read_csv('{}[0-15].csv'.format(i))[j:] for i,j in {'GALAXY':nG,'STAR':nS,'QSO':nQ}.items()]
frames2= [pd.read_csv('{}[15-24].csv'.format(i)) for i in ['GALAXY','STAR','QSO']]
    
x_te1= pd.concat([frames2[0],frames2[1],frames2[2],frames1[0],frames1[1],frames1[2]])
x_te= x_te1.sample(frac=1).reset_index(drop=True)

In [ ]:
y_tr= []
for i in range(len(x_tr)): 
    c= x_tr['class'][i] 
    y_tr.append([c=='STAR',c=='GALAXY',c=='QSO']) 
y_tr= np.array(y_tr).astype(int) 
 
x_tr= np.array(x_tr.drop(columns=['class'])) 
 
y_te= [] 
for i in range(len(x_te)): 
    c= x_te['class'][i] 
    y_te.append([c=='STAR',c=='GALAXY',c=='QSO']) 
y_te= np.array(y_te).astype(int) 
 
x_te= np.array(x_te.drop(columns=['class']))

In [ ]:
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(x_tr.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(3, activation='sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(x_tr,y_tr,epochs=20,batch_size=300,verbose=1,shuffle=False)

In [ ]:
model.evaluate(x_te,y_te)

In [ ]:
y_pred = model.predict(x_te)
y_true = y_te.argmax(axis=1)
y_pred = y_pred.argmax(axis=1)

In [ ]:
cm = pd.DataFrame(confusion_matrix(y_true, y_pred, normalize='pred'),
                      index=['STAR','GALAXY','QSO'], columns=['STAR','GALAXY','QSO']))
sns.heatmap(cm, annot=True, cmap=plt.cm.Blues)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()